In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from tqdm.auto import tqdm

from torch import nn

# ===========================
# CONFIG
# ===========================
class Config:
    DATA_DIR = Path("/kaggle/input/nfl-big-data-bowl-2026-prediction/")
    MODEL_DIR = Path("/kaggle/input/temporal2") 
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    WINDOW_SIZE = 10
    MAX_FUTURE_HORIZON = 94
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3

config = Config()

test_input = pd.read_csv(config.DATA_DIR / "test_input.csv")
test_template = pd.read_csv(config.DATA_DIR / "test.csv")


In [ ]:
def height_to_feet(height_str):
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0

def add_advanced_features(df):
    """Enhanced feature engineering from Notebook 1"""
    print("Adding advanced features...")
    df = df.copy()
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']

    # Distance Rate Features
    if 'distance_to_ball' in df.columns:
        df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
        df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
        df['time_to_intercept'] = (df['distance_to_ball'] /
                                    (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)

    # Target Alignment Features
    if 'ball_direction_x' in df.columns:
        df['velocity_alignment'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
        df['velocity_perpendicular'] = (
            df['velocity_x'] * (-df['ball_direction_y']) +
            df['velocity_y'] * df['ball_direction_x']
        )
        if 'acceleration_x' in df.columns:
            df['accel_alignment'] = (
                df['acceleration_x'] * df['ball_direction_x'] +
                df['acceleration_y'] * df['ball_direction_y']
            )

    # Multi-Window Rolling
    for window in [3, 5, 10]:
        for col in ['velocity_x', 'velocity_y', 's', 'a']:
            if col in df.columns:
                df[f'{col}_roll{window}'] = df.groupby(gcols)[col].transform(
                    lambda x: x.rolling(window, min_periods=1).mean()
                )
                df[f'{col}_std{window}'] = df.groupby(gcols)[col].transform(
                    lambda x: x.rolling(window, min_periods=1).std()
                ).fillna(0)

    # Extended Lag Features
    for lag in [4, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y']:
            if col in df.columns:
                df[f'{col}_lag{lag}'] = df.groupby(gcols)[col].shift(lag).fillna(0)

    # Velocity Change Features
    if 'velocity_x' in df.columns:
        df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
        df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
        df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
        df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
        df['direction_change'] = df['direction_change'].apply(
            lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
        )

    # Field Position Features
    df['dist_from_left'] = df['y']
    df['dist_from_right'] = 53.3 - df['y']
    df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
    df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])

    # Role-Specific Features
    if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
        df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
        df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
    if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
        df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']

    # Time Features
    df['frames_elapsed'] = df.groupby(gcols).cumcount()
    df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
        lambda x: x / (x.max() + 1)
    )

    return df

def prepare_combined_features(input_df, output_df=None, test_template=None, is_training=True, window_size=10):
    """COMBINED: Advanced features + enhanced preprocessing"""
    print(f"Preparing COMBINED sequences (window_size={window_size})...")

    input_df = input_df.copy()


    # BASIC FEATURES

    input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)

    # Enhanced motion features (From Notebook 2)
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    o_rad = np.deg2rad(input_df['o'].fillna(0))

    input_df['velocity_x'] = input_df['s'] * np.sin(dir_rad)
    input_df['velocity_y'] = input_df['s'] * np.cos(dir_rad)
    input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
    input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)
    input_df['orientation_x'] = np.sin(o_rad)
    input_df['orientation_y'] = np.cos(o_rad)

    # Enhanced roles
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    input_df['is_rusher'] = (input_df['player_role'] == 'Pass Rusher').astype(int)

    # Field position (enhanced)
    input_df['field_x_norm'] = (input_df['x'] - Config.FIELD_X_MIN) / (Config.FIELD_X_MAX - Config.FIELD_X_MIN)
    input_df['field_y_norm'] = (input_df['y'] - Config.FIELD_Y_MIN) / (Config.FIELD_Y_MAX - Config.FIELD_Y_MIN)

    # Physics features
    mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
    input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
    input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
    input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)

    # Ball features
    if 'ball_land_x' in input_df.columns:
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed'] = (
            input_df['velocity_x'] * input_df['ball_direction_x'] +
            input_df['velocity_y'] * input_df['ball_direction_y']
        )

    # Sort for temporal features
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']

    # Enhanced temporal features
    for lag in [1, 2, 3, 5]:
        input_df[f'x_lag{lag}'] = input_df.groupby(gcols)['x'].shift(lag)
        input_df[f'y_lag{lag}'] = input_df.groupby(gcols)['y'].shift(lag)
        input_df[f'velocity_x_lag{lag}'] = input_df.groupby(gcols)['velocity_x'].shift(lag)
        input_df[f'velocity_y_lag{lag}'] = input_df.groupby(gcols)['velocity_y'].shift(lag)
        input_df[f's_lag{lag}'] = input_df.groupby(gcols)['s'].shift(lag)

    # Multiple EMA smoothing (
    for alpha in [0.1, 0.3, 0.5]:
        input_df[f'velocity_x_ema_{alpha}'] = input_df.groupby(gcols)['velocity_x'].transform(
            lambda x: x.ewm(alpha=alpha, adjust=False).mean()
        )
        input_df[f'velocity_y_ema_{alpha}'] = input_df.groupby(gcols)['velocity_y'].transform(
            lambda x: x.ewm(alpha=alpha, adjust=False).mean()
        )


    # ADVANCED FEATURES

    input_df = add_advanced_features(input_df)


    # COMBINED FEATURE LIST

    feature_cols = [
        # Core tracking (8)
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id',
        'ball_land_x', 'ball_land_y',

        # Player attributes (2)
        'player_height_feet', 'player_weight',

        # Enhanced motion (7)
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'orientation_x', 'orientation_y',
        'kinetic_energy',

        # Roles (6)
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer', 'is_rusher',

        # Field position (6)
        'field_x_norm', 'field_y_norm',
        'dist_from_sideline', 'dist_from_endzone',
        'distance_to_sideline', 'distance_to_endzone',

        # Ball interaction (5)
        'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',

        # Enhanced temporal (20)
        'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1', 's_lag1',
        'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2', 's_lag2',
        'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3', 's_lag3',
        'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5', 's_lag5',

        # Multiple EMAs (6)
        'velocity_x_ema_0.1', 'velocity_y_ema_0.1',
        'velocity_x_ema_0.3', 'velocity_y_ema_0.3',
        'velocity_x_ema_0.5', 'velocity_y_ema_0.5',

        # Advanced features (20+)
        'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
        'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
        'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
        'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
        'frames_elapsed', 'normalized_time',

        # Rolling features (selective)
        'velocity_x_roll5', 'velocity_y_roll5', 's_roll5', 'a_roll5',
        'velocity_x_std5', 'velocity_y_std5', 's_std5', 'a_std5',
    ]

    # Filter to existing columns
    feature_cols = [c for c in feature_cols if c in input_df.columns]
    print(f"Using {len(feature_cols)} COMBINED features")


    # CREATE SEQUENCES

    input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])

    target_rows = output_df if is_training else test_template
    target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()

    sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []

    for _, row in tqdm(target_groups.iterrows(), total=len(target_groups)):
        key = (row['game_id'], row['play_id'], row['nfl_id'])

        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue

        input_window = group_df.tail(window_size)

        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)

        # Enhanced imputation
        input_window = input_window.ffill().bfill()
        input_window = input_window.fillna(group_df.mean(numeric_only=True))

        seq = input_window[feature_cols].values

        if np.isnan(seq).any():
            if is_training:
                continue
            seq = np.nan_to_num(seq, nan=0.0)

        sequences.append(seq)

        if is_training:
            out_grp = output_df[
                (output_df['game_id']==row['game_id']) &
                (output_df['play_id']==row['play_id']) &
                (output_df['nfl_id']==row['nfl_id'])
            ].sort_values('frame_id')

            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']

            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y

            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)

        sequence_ids.append({
            'game_id': key[0],
            'play_id': key[1],
            'nfl_id': key[2],
            'frame_id': input_window.iloc[-1]['frame_id']
        })

    print(f"Created {len(sequences)} sequences")

    if is_training:
        return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids, feature_cols
    return sequences, sequence_ids, feature_cols

In [ ]:
from torch import nn

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride=1, dilation=1, padding=0, dropout=0.2):
        super().__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        res = x if self.downsample is None else self.downsample(x)
        return nn.functional.relu(out + res)
class EnhancedSeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.horizon = horizon

        self.gru = nn.GRU(input_dim, 192, num_layers=3, batch_first=True, dropout=0.2)

        self.conv1d = nn.Sequential(
            TemporalBlock(192, 128, kernel_size=3, padding=1),
            nn.GELU(),
            TemporalBlock(128, 128, kernel_size=5, padding=2),
            nn.GELU(),
        )

        self.pool_ln = nn.LayerNorm(192)
        self.pool_attn = nn.MultiheadAttention(192, num_heads=8, batch_first=True, dropout=0.1)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 192))

        self.head = nn.Sequential(
            nn.Linear(192 + 128, 256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(128, horizon * 2)
        )

        self.initialize_weights()

    def initialize_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.GRU):
                for name, param in module.named_parameters():
                    if 'weight' in name:
                        nn.init.orthogonal_(param)
                    elif 'bias' in name:
                        nn.init.constant_(param, 0)

    def forward(self, x):
        h, _ = self.gru(x)

        h_conv = self.conv1d(h.transpose(1, 2)).transpose(1, 2)
        h_conv_pool = h_conv.mean(dim=1)

        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        h_norm = self.pool_ln(h)
        ctx, _ = self.pool_attn(q, h_norm, h_norm)
        ctx = ctx.squeeze(1)

        combined = torch.cat([ctx, h_conv_pool], dim=1)

        out = self.head(combined)
        out = out.view(B, 2, self.horizon)

        out = torch.cumsum(out, dim=2)

        return out[:, 0, :], out[:, 1, :]

In [ ]:

test_sequences, test_ids, feature_cols = prepare_combined_features(
    test_input, test_template=test_template, is_training=False, window_size=config.WINDOW_SIZE
)

INPUT_DIM = len(feature_cols)
print(f"✅ Detected INPUT_DIM = {INPUT_DIM}")

X_test = np.array(test_sequences, dtype=object)
x_last = np.array([s[-1, 0] for s in X_test])
y_last = np.array([s[-1, 1] for s in X_test])


models, scalers = [], []
for fold in range(1, 6):
    model_path = config.MODEL_DIR / f"best_model_fold{fold}.pt"

    
    checkpoint = torch.load(model_path, map_location=config.DEVICE, weights_only=False)
    
    
    model = EnhancedSeqModel(input_dim=INPUT_DIM, horizon=config.MAX_FUTURE_HORIZON)
    model.load_state_dict(checkpoint["model_state"])
    model.to(config.DEVICE)
    model.eval()
    models.append(model)

    # スケーラー復元
    scaler = StandardScaler()
    scaler.mean_ = np.array(checkpoint["scaler_mean"])
    scaler.scale_ = np.array(checkpoint["scaler_scale"])
    scalers.append(scaler)

    print(f"✅ Loaded: {model_path.name}, with scaler")


all_dx, all_dy = [], []

for model, scaler in zip(models, scalers):
    
    X_scaled = np.stack([scaler.transform(s) for s in X_test])
    X_tensor = torch.tensor(X_scaled.astype(np.float32)).to(config.DEVICE)
    
    with torch.no_grad():
        dx, dy = model(X_tensor)
        all_dx.append(dx.cpu().numpy())
        all_dy.append(dy.cpu().numpy())

ens_dx = np.mean(all_dx, axis=0)
ens_dy = np.mean(all_dy, axis=0)


rows = []
H = ens_dx.shape[1]

for i, sid in enumerate(test_ids):
    fids = test_template[
        (test_template['game_id'] == sid['game_id']) &
        (test_template['play_id'] == sid['play_id']) &
        (test_template['nfl_id'] == sid['nfl_id'])
    ]['frame_id'].sort_values().tolist()

    for t, fid in enumerate(fids):
        tt = min(t, H - 1)
        px = np.clip(x_last[i] + ens_dx[i, tt], config.FIELD_X_MIN, config.FIELD_X_MAX)
        py = np.clip(y_last[i] + ens_dy[i, tt], config.FIELD_Y_MIN, config.FIELD_Y_MAX)
        rows.append({
            'id': f"{sid['game_id']}_{sid['play_id']}_{sid['nfl_id']}_{fid}",
            'x': float(px),
            'y': float(py)
        })

submission = pd.DataFrame(rows)
submission['id'] = submission['id'].astype(str)
submission['x'] = submission['x'].astype(float)
submission['y'] = submission['y'].astype(float)

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved:", submission.shape)

In [ ]:
submission.head()